# Lösen einer Gewöhnlichen Differential Gleichung (GDL)

Eine der einfachsten Methoden, um eine *GDL* zu lösen, ist, das sogenannte **Euler-vorwärts-Verfahren** oder **explicite Euler-Verfahren**. Dabei benutzen wir die Informationen aus dem aktuellen Zeitschritt, um die Informationen zum neuen Zeitschritt zu berechnen. Dieses Verfahren wollen wir an einem bestimmten ein-dimesionalen Problem anwenden. 

## Das Problem

Eine, im 1-D Raum, relative einfache *GDL* ist die Partikelpfadgleichung, gegeben durch:

$$
\frac{\partial{x}}{\partial{t}} = f(x) = v_{x}, \tag{1}
$$

wobei $x$ die Position des Partikels, $t$ die Zeit, und $v_{x}$ die horizontale Geschwindigkeit darstellt. Die Gleichung beschreibt die Änderung der Position eines Partikels *P* mit der Zeit unter der Annahme einer konstanten Geschwindigkeit. 

Bevor wir versuchen, das Problem numerisch zu lösen, lasst uns erst einmal die Anfangssituation darstellen. Dazu benötigen wir die folgenden Module in Julia (```Plots```):

In [ ]:
using Plots

Nun definieren wir die Konstanten für unser Problem (Geometrie, Zeit, Physik): 

In [ ]:
# Plot Parameter -------------------------------------------------------- %
#   Definiert ob die Abbildung als Animation gespeichert wird, 
#   0 - nein, 1 - ja
save_fig    =   1    
# ----------------------------------------------------------------------- %
# Geometrische Konstanten ----------------------------------------------- %
xmin        =   -10.0                               # Minimum [ m ]
xmax        =   10.0                               # Maximum [ m ]
# ----------------------------------------------------------------------- %
# Zeit Parameter -------------------------------------------------------- %
tmin        =   0.0                             # Anfangszeit [ s ]
tmax        =   60                               # Maximale Zeit [ s ]
dt          =   0.5                               # Zeitschrittlänge [ s ]
nt          =   Int((tmax - tmin) / dt) + 1     # Anzahl der Zeitschritte 
Zeit        =   0.0
# ----------------------------------------------------------------------- %
# Anfangswert ----------------------------------------------------------- %
vx          =   0.5                               # Geschwindigkeit [ m/s ]
Px          =   0                               # Anfangsposition 
# ----------------------------------------------------------------------- %

### Visualizierung
Um unsere Abbildungen in einer Animation speichern zu können, müssen wir den Ordner und den Namen der Datei festlegen: 

In [ ]:
# Animationssettings ---------------------------------------------------- %
path        =   string("./Results/")
anim        =   Plots.Animation(path, String[] )
filename    =   "01_1D_Euler"

Jetzt wollen wir das Partikel an seiner Anfangsposition darstellen (```scatter,annotate ```): 

In [ ]:
p = scatter([Px], [0], label="", xlims=(xmin, xmax), ylims=(-1, 1),
            xlabel="x", ylabel="y", title="Partikelpfad 1D",            
            marker=:circle, markersize=8,
            markercolor=:black, markerstrokecolor=:red)    
annotate!(p, -0.75, 0.75, text("Zeit = $Zeit", :black, 12)) 
display(p)

Als nächstes wollen wir die *GDL* lösen.

## Die Lösung

Die partielle Differentialgleichung können wir, wie in der Vorlesung besprochen, durch die Schreibweise der *finiten Differenzen* approximieren. Dabei verwenden wir eine Approximation **vorwärts** in der Zeit und **vorwärts** im Raum. Die Partikelpfadgleichung ist dann gegeben durch: 

$$
\frac{\Delta{x}}{\Delta{t}} = v_x, \tag{2}
$$

wobei $\Delta{}$ das totale oder absolute Differential darstellt und ausgeschrieben gegeben durch:  

$$
\frac{x_2-x_1}{t_2-t_1}=v_x, \tag{3}
$$

wobei $x_1$ und $x_2$ die Position des Partikel zum aktuellen und neuen Zeitpunkt und $t_1$ und $t_2$ die Zeit zum aktuellen und neuen Zeitpunkt darstellt. 

Nun können wir die Gleichung nach der Unbekannten $x_2$ auflösen: 

$$
x_2 = v_x \Delta{t} + x_1. \tag{4}
$$

Diese Gleichung müssen wir nun für jeden Zeitschritt lösen und können dann die Lösung unseres Problems, die Position des Partikels mit der Zeit, für jeden Zeitschritt graphisch darstellen. Dazu müssen wir diese Schritte nacheinander in einer Schleife über die Zeit numerisch berechnen und darstellen. 

In [ ]:
# Numerische Loesung der GDL -------------------------------------------- %
for it = 1:nt
    Zeit = (it - 1) * dt
    println("Time: ", Zeit)
    if it > 1
        Px = vx * dt + Px
    end
    if Px > xmax
        Px = Px - xmax + xmin
    end
    if Px < xmin
        Px = Px + xmax - xmin
    end    
    p = scatter([Px], [0], label="", xlims=(xmin, xmax), ylims=(-1, 1),
            xlabel="x", ylabel="y", title="Partikelpfad 1D",            
            marker=:circle, markersize=8,
            markercolor=:black, markerstrokecolor=:red)    
    annotate!(p, -0.75, 0.75, text("Zeit = $Zeit", :black, 12)) 

    Plots.frame(anim)

end

Um die Abbildungen in einer Gifanimation zu speichern, müssen wir noch das entsprechende Gif erzeugen: 

In [ ]:
if save_fig == 1
    # Write the frames to a GIF file
    Plots.gif(anim, string( path, filename, ".gif" ), fps = 15)
end
foreach(rm, filter(startswith(string(path,"00")), readdir(path,join=true)))